In [3]:
import pandas as pd

# Chapter 6

## Data Loading, Storage, and File Formats

## 6.1 Reading and Writing Data in Text Format

pandas has a number of functions for reading tabular data, but will most likely use read_csv and read_table

optional arguments for these functions include: indexing, type inference and data conversion, datetime parsing, iterating, and unclean data issues

handling dates and other custom types can require extra effort. start with a small comma-separated (CSV) file:

In [1]:
!cat examples/ex1.csv

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [4]:
df = pd.read_csv('examples/ex1.csv')

In [5]:
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [6]:
pd.read_table('examples/ex1.csv', sep=',') #used read-table and specified the deliminator

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [7]:
!cat examples/ex2.csv #file will not always have a header row

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [8]:
pd.read_csv('examples/ex2.csv', header=None) #allow pandas to assign default column names

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [9]:
pd.read_csv('examples/ex2.csv', names=['a', 'b', 'c', 'd', 'message']) #assign column names

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [10]:
names = ['a', 'b', 'c', 'd', 'message']

In [11]:
pd.read_csv('examples/ex2.csv', names=names, index_col='message') #setting message column to be the index

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


if you want to form a hierarchical index from multiple columns, pass a list of column numbers or names:

In [12]:
!cat examples/csv_mindex.csv

key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16


In [13]:
parsed = pd.read_csv('examples/csv_mindex.csv', index_col=['key1', 'key2'])

In [14]:
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

in some cases a table might not have a fixed delimiter, using whitespace or some other pattern to seaparate fields. for example:


In [15]:
list(open('examples/ex3.txt'))

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100491\n']

the fields here are separated by a variable amount of whitespace. in these cases, you can pass a regular expression as a delimiter for read-table. use the reg:ex \s+

In [17]:
result = pd.read_table('examples/ex3.txt', sep='\s+')

In [18]:
result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


In [20]:
!cat examples/ex4.csv


# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [22]:
pd.read_csv('examples/ex4.csv', skiprows=[0, 2, 3]) #skipping rows

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


handling missing values- by default, pandas uses a set of commonly occuring sentinels such as NA and NULL

In [23]:
!cat examples/ex5.csv

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo

In [24]:
result = pd.read_csv('examples/ex5.csv')

In [25]:
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [26]:
pd.isnull(result)

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


the na_values option can take either a list or set of strings to consider missing values:

In [27]:
result = pd.read_csv('examples/ex5.csv', na_values=['Null'])

In [28]:
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [31]:
sentinels = {'message': ['foo', 'NA'], 'something': ['two']} #differente NA sentinels can be specified for each column in a dict: